In [1]:
import matplotlib.pyplot as plt
import numpy as np
import helper
import time
import datetime
import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import pandas as pd
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.metrics import classification_report
from tqdm import tqdm
import math
from torcheval.metrics import BinaryAccuracy
import os
import timm
import segmentation_models_pytorch as smp
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from timm import create_model
import cv2
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
batch_size=1
img_size=1024
class_list=['NT_epithelial','NT_immune','NT_stroma','TP_in_situ','TP_invasive']
class_weight=[2,0,1,3,4]
tf = ToTensor()
topilimage = torchvision.transforms.ToPILImage()
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/torch/onnx/_internal/_beartype.py:36: UserWarning: unhashable type: 'list'
  warnings.warn(f"{e}")


In [2]:
img_path='../../data/area_segmentation/BR_class/image/'
img_list=glob(img_path+'NIA6_R_BRID*.jpeg')
mask_list=[]
train_img_list,test_img_list=train_test_split(img_list,test_size=0.2,random_state=42)
test_image=torch.zeros((len(test_img_list),3,img_size,img_size))
test_mask=torch.zeros((len(test_img_list),len(class_list),img_size,img_size),dtype=torch.float32)    

for i in tqdm(range(len(test_img_list))):
    test_image[i] = tf(Image.open(test_img_list[i]))
    for j in range(len(class_list)):
        tf_mask=tf(Image.open(test_img_list[i].replace('/image', '/'+class_list[j])).convert('L'))
        test_mask[i,j]=tf_mask

100%|██████████| 90/90 [00:06<00:00, 12.98it/s]


In [3]:
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list
        self.label = label_list
        
    def trans(self,image,label):
        if random.random() > 0.5:
            transform = transforms.RandomHorizontalFlip(1)
            label = transform(label)
            image = transform(image)
            
        if random.random() > 0.5:
            transform = transforms.RandomVerticalFlip(1)
            label = transform(label)
            image = transform(image)
            
        return image,label
    
    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        image_path,label_path = self.img_path[idx],self.label[idx]
        
        return image_path, label_path

test_dataset = CustomDataset(test_image, test_mask)

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [4]:

NT_epithelial_model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset) 
    ).to(device)
NT_immune_model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset) 
    ).to(device)
NT_stroma_model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset) 
    ).to(device)
TP_in_situ_model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset) 
    ).to(device)
TP_invasive_model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset) 
    ).to(device)
def dice_loss(pred, target, num_classes=2):
    smooth = 1e-6
    dice_per_class = torch.zeros((len(pred),num_classes)).to(pred.device)
    pred=F.softmax(pred,dim=1)
    for i in range(len(pred)):
        for class_id in range(num_classes):
            pred_class = pred[i, class_id, ...]
            target_class = target[i, class_id, ...]
            
            intersection = torch.sum(pred_class * target_class)
            A_sum = torch.sum(pred_class * pred_class)
            B_sum = torch.sum(target_class * target_class)
            dice_per_class[i,class_id] =(2. * intersection + smooth) / (A_sum + B_sum + smooth)

    return 1-dice_per_class.mean()
# summary(model,(batch_size,3,img_size,img_size))

NT_epithelial_model.load_state_dict(torch.load('../../model/areaSeg/BR_'+class_list[0]+'.pt',map_location=device))
NT_immune_model.load_state_dict(torch.load('../../model/areaSeg/BR_'+class_list[1]+'.pt',map_location=device))
NT_stroma_model.load_state_dict(torch.load('../../model/areaSeg/BR_'+class_list[2]+'.pt',map_location=device))
TP_in_situ_model.load_state_dict(torch.load('../../model/areaSeg/BR_'+class_list[3]+'.pt',map_location=device))
TP_invasive_model.load_state_dict(torch.load('../../model/areaSeg/BR_'+class_list[4]+'.pt',map_location=device))
model_list=[NT_epithelial_model,NT_immune_model,NT_stroma_model,TP_in_situ_model,TP_invasive_model]

/tmp/ipykernel_649088/4153989047.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  NT_epithelial_model.load_state_dict(torch.load('../../model/areaSeg/BR_'+class_list[0]+

In [21]:

topilimage = torchvision.transforms.ToPILImage()
k = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
train_loss_list=[]
val_loss_list=np.zeros((len(test_dataloader),5))
train_acc_list=[]
val_acc_list=[]
MIN_loss=5000
metrics = defaultdict(float)
for i in range(len(model_list)):
    model_list[i].eval()
count=0
val_running_loss=0.0
acc_loss=0
with torch.no_grad():
    for x, y in test_dataloader:
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        predict=torch.zeros((1,5,1024,1024)).to(device)
        for i in class_weight:
            model=model_list[i]
            predict1 = model(x).to(device)
            cost = dice_loss(predict1, y[:,i]).cpu().numpy()
            val_loss_list[count-1,i]=1-cost
            predict_index=torch.where(F.softmax(predict1, dim=1).argmax(dim=1)==1)[0]
            for j in range(5):
                predict[0,j][predict_index]=0.
            predict[0,i][predict_index]=1.
            if i==4:
                print(F.softmax(predict1, dim=1).argmax(dim=1).max())
        y = y.to('cpu')
        x=x.to('cpu')
        
        if count%20==0:
            pred_softmax=np.array(predict.cpu())
            mask=np.zeros((img_size,img_size,3))
            mask[...,0]=cv2.morphologyEx(pred_softmax[0][0], cv2.MORPH_OPEN, k)
            mask[...,1]=cv2.morphologyEx(pred_softmax[0][1], cv2.MORPH_OPEN, k)
            mask[...,2]=cv2.morphologyEx(pred_softmax[0][2], cv2.MORPH_OPEN, k)
            mask[...,1]+=cv2.morphologyEx(pred_softmax[0][3], cv2.MORPH_OPEN, k)
            mask[...,2]+=cv2.morphologyEx(pred_softmax[0][3], cv2.MORPH_OPEN, k)
            mask[...,0]+=cv2.morphologyEx(pred_softmax[0][4], cv2.MORPH_OPEN, k)
            mask[...,1]+=cv2.morphologyEx(pred_softmax[0][4], cv2.MORPH_OPEN, k)
            label=np.zeros((img_size,img_size,3))
            label[...,0]=np.array(y[0][0])
            label[...,1]=np.array(y[0][1])
            label[...,2]=np.array(y[0][2])
            label[...,1]+=np.array(y[0][3])
            label[...,2]+=np.array(y[0][3])
            label[...,0]+=np.array(y[0][4])
            label[...,1]+=np.array(y[0][4])
            img=topilimage(x[0])
            overlay=np.array(img)*0.7+np.array(mask)*0.3
            overlay=Image.fromarray(overlay.astype('uint8'))
            fig = plt.figure(figsize=(20,30))
            rows = 1
            cols = 4
            ax1 = fig.add_subplot(rows, cols, 1)
            ax1.imshow(img)
            ax1.set_title('image')
            ax1.axis("off")
            
            ax2 = fig.add_subplot(rows, cols, 2)
            ax2.imshow(label)
            ax2.set_title('label')
            ax2.axis("off")
            
            ax3 = fig.add_subplot(rows, cols, 3)
            ax3.imshow(mask)
            ax3.set_title('predict')
            ax3.axis("off")
            
            ax4 = fig.add_subplot(rows, cols, 4)
            ax4.imshow(overlay)
            ax4.set_title('overlay')
            ax4.axis("off")
            plt.show()
           

print('batch size= 4')
print('image size= 224')
print('learning rate= 0.0001')

tensor(0, device='cuda:6')
tensor(0, device='cuda:6')
tensor(0, device='cuda:6')
tensor(0, device='cuda:6')
tensor(0, device='cuda:6')
tensor(0, device='cuda:6')
tensor(0, device='cuda:6')
tensor(0, device='cuda:6')


KeyboardInterrupt: 

In [6]:
from shapely.geometry import Polygon, MultiPolygon
import cv2
def binary_mask_to_polygon(binary_mask):
    # binary_mask는 2차원 numpy array여야 합니다.
    # Contours를 찾습니다.
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    polygons = []
    for contour in contours:
        # 각 contour를 polygon으로 변환
        if len(contour) >= 3:  # 유효한 polygon을 만들기 위해서 최소한 3개의 점이 필요합니다.
            poly = Polygon(shell=[(point[0][0], point[0][1]) for point in contour])
            polygons.append(poly)
    
    if len(polygons) > 1:
        # 여러 개의 polygon이 있을 경우 MultiPolygon으로 변환
        return polygons
    elif len(polygons) == 1:
        return polygons[0]
    else:
        return None
    
poly=binary_mask_to_polygon(np.array(mask)[...,1])
polygon_arrays = []
for polygon in poly.geoms:
    exterior_coords = np.array(polygon.exterior.coords)
    polygon_arrays.append(exterior_coords)

# 변환된 numpy 배열 출력
for i, array in enumerate(polygon_arrays):
    print(f"Polygon {i+1} coordinates:")
    print(array)

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/contours.cpp:195: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'


In [ ]:
for polygon in poly:
    print(polygon)

In [ ]:
from skimage.draw import polygon2mask
mask1=np.zeros((1024,1024))
for i in range(len(polygon_arrays)):S
    coordinate=np.array(polygon_arrays[i])
    coordinate[:,[0, 1]]=coordinate[:,[1, 0]]
    mask_temp=polygon2mask((1024,1024), coordinate)
    mask1+=mask_temp
ax=plt.figure(figsize=(10,10))
ax.add_subplot(1,3,1)
plt.imshow(img)
ax.add_subplot(1,3,2)
plt.imshow(mask1)
ax.add_subplot(1,3,3)
img1=np.array(img)
img1[...,0]=img1[...,0]*0.8+mask1*255*0.2
plt.imshow(img1)

In [ ]:
len(polygon_arrays)

In [ ]:
poly